__Ejercicio 1:__ Crear la libreria herramientas dentro de su carpeta de tareas, en su repositorio. Dentro deberán poner los métodos de Newton que construyeron, los métodos de integración, Euler independiente de dimensión (implícito y explícito)y Runge-Kutta independiente de dimensión. Recuerden que las rutinas deben funcionar solo con sus argumentos, no debe haber variables globales, todo lo que necesite la función para trabajar debe ser un argumento o una variable definida internamente.
* Las rutinas tienen que llevar su documentación.
* Crearán el notebook de la __tarea 15 (para el Martes 17)__ y en el llamarán a su libreria con el comando `import`.
* Deberán demostrar que sus rutinas funcionan bien, no se necesitan tests extensos, basta con que muestren que si _corren_ las rutinas.

In [1]:
__precompile__() # Este comando es para que julia precompile el paquete

module herramientas

#Programas método Newton [Tarea 4]:

export metodo_newton
"""documentación del método de Newton"""
function metodo_newton(f,df,x0)
    x=x0
    for i in 1:20
       x=x-f(x)/df(x) 
    end
    return x
end

export metodo_newton_numerico
"""documentación del método de Newton"""
function metodo_newton_numerico(f,x0) #Definimos como al principio
    x = x0
    h = 0.1
    for i in 1:100
        x = x-(f(x))/((f(x+h)-f(x))/(h));     #A partir de aquí, uno se da la idea de que existe un cierto error (Tomar esto como hipotesis)
    end
    return x
end;

export metodo_newton_lista
"""documentación del método de Newton tipo lista"""
function metodo_newton_lista(f,df,inits)
    list=zeros(length(inits))  #Si tenemos n particiones se hace una lista con n elementos
    x = 0.0
    for i in 1:length(inits)   #Aplicamos metodo de newton para cada elemento de nuestro linspace
        x = inits[i]
        for n in 1:200         #Ciclo for, con 200 iteraciones (Este es el epsilon para el ejercicio siguiente)
            x = x-(f(x)/df(x))        
        end
        list[i] = x            #Una lista con iteraciones finales
    end
    return list                #Return hasta que ya haya obtenido las n raices
end;

export metodo_newton_epsilon
"""documentación de metodo de newton epsilon"""
function metodo_newton_epsilon(f,df,inits)
    t = []                                  #Hacemos t un vector vacío
    epsilon = 0.0000001                     #Nuestra epsilon
    list = metodo_newton_lista(f,df,inits)  #Por comodidad, llamamos a la funcion de la lista como lista
    push!(t,list[1])                        #El primer push, para que pueda empezar el ciclo for de i:
    for i in 1:length(t)                    #Por cada elemento de t, hacer esto. Notese que con el push se pueden agregar nuevos elementos
        for n in 1:length(list)             #Ciclo for anidado, ahora sí todos los elementos de la lista de raices
            if abs(t[i]-list[n]) > epsilon  #Comparamos todos los elementos de la lista por solo 1 de t
                push!(t,list[n])            #Si el abs de eso es mayor a epsilon, nos indica que es una raiz diferente.
            end                             #Con el push se agrega un elemento nuevo a t y se repite el primer ciclo for y despues el segundo, de tal manera que empezamos con 1 elemento en t, se compara ese solo elemento con toda la lista: se encuentra una raiz nueva, se agrega a t teniendo 2 elementos y cada uno de esos elementos puede compararse con toda la lista hasta obtener raices diferentes y asi consecutivamente.
        end
        return t                           #Que nos de la lista de raices diferentes                          
    end
end

export metodo_newton_varias_condiciones_iniciales

export integracion_rectangulo
"""documentación de integración por método del rectángulo"""
function integracion_rectangulo(f,a,b,numparticiones)        #Copy-paste de las funciones hechas en tarea 6
    anchorec = (b-a)/numparticiones                     
    s = []                                             
    suma = 0.0                                          
    area = 0.0                                          
    ainit = a                                          
    for i in 1:numparticiones                           
        b = (anchorec)*(i) +ainit                       
        a = ainit+(anchorec)*(i-1)              
        area = (b-a)*(f((b+a)/2))                       
        push!(s,area)                                   
    end
    suma = sum(s,1:numparticiones)                      
    return suma
end;

export integracion_trapecio
"""documentación de integración por método de trapecio"""
function integracion_trapecio(f,a,b,numparticiones)
    anchorec = (b-a)/numparticiones                     
    s = []
    suma = 0.0                    
    area = 0.0                                          
    ainit = a                       
    for i in 1:numparticiones                         
        b = (anchorec)*(i) +ainit                      
        a = ainit+(anchorec)*(i-1)              
        area = (b-a)*((f(a)+f(b))/2)                      
        push!(s,area)
    end
    suma = sum(s,1:numparticiones)
    return suma
end;

export integracion_simpson
"""documentación de integración por método de Simpson"""
function integracion_simpson(f,a,b,numparticiones)
    anchorec = (b-a)/numparticiones                     #practicamente lo mismo que el metodo del rectangulo
    s = []
    suma = 0.0                    
    area = 0.0                                          
    ainit = a                       
    for i in 1:numparticiones                         
        b = (anchorec)*(i) +ainit                      
        a = ainit+(anchorec)*(i-1)              
        area = ((b-a)/6)*((f(a)+4f((a+b)/2)+f(b)))      #p.d. regla de simpson
        push!(s,area)
    end
    suma = sum(s,1:numparticiones)
    return suma
end;

export euler
"""documentación del método de Euler explícito"""
function euler(f,list,x0)
     x = x0
     h = list[2]-list[1]
     listx=[]
     push!(listx,x)
     for i in 2:length(list)
        t = i*h
        x = x + f(x,t)*h
        push!(listx,x) 
     end
     return listx
    end;

export euler_implicito
"""documentación del método de Euler implicito"""
function euler_implicito(f,list,x0)
     x = x0
     h = list[2]-list[1]
     listx=[]
     push!(listx,x)
     for i in 2:length(list)
        t = i*h
        t2 = i*h+h
        t_final = h*length(list)
        g(u) = u-x-f(x,t)*h
        x = metodo_newton_numerico(g,x)
        push!(listx,x) 
     end
     return listx
    end;

export euler_pm
"""documentación del método de Euler p.m."""
function euler_pm(f,list,x0)
     x = x0
     h = list[2]-list[1]
     listx=[]
     push!(listx,x)
     for i in 2:length(list)
        t = i*h
        t2 = i*h+h/2
        x = x + +h*f(x+f(x,t)*h/2,t2)
        push!(listx,x) 
     end
     return listx
    end;

export runge_kutta
"""documentación del método de Runge-Kutta"""
function runge_kutta(f,list,x0)
     x = x0
     h = list[2]-list[1]
     listx=[]
     push!(listx,x)
     for i in 2:length(list)
        t = i*h
        t2 = i*h+h/2
        #Se actualizan las "kutas":
        k1 = f(x,t)
        k2 = f(x+(h/2)*k1,t2)
        k3 = f(x+(h/2)*k2,t2)
        k4 = f(x+h*k3,t)
        x = x+(h/6)*(k1+2*k2+2*k3+k4) #Por definicion Runge_kutta orden 4
        push!(listx,x) 
     end
     return listx
end;


end

herramientas

In [2]:
import herramientas

### Test para método de Newton:

In [3]:
function f(x);
    x^3-x^2-x-1
end;

function df(x);
    3x^2-x^2-1
end;

In [4]:
herramientas.metodo_newton(f,df,1)

1.8392867552141612

In [5]:
herramientas.metodo_newton_lista(f,df,linspace(0,10,3))

3-element Array{Float64,1}:
 1.83929
 1.83929
 1.83929

In [6]:
herramientas.metodo_newton_epsilon(f,df,linspace(0,10,3))

1-element Array{Any,1}:
 1.83929

### Test para métodos de integración:

In [7]:
herramientas.integracion_rectangulo(f,0,1,100)

1-element Array{Float64,1}:
 -1.58334

In [8]:
herramientas.integracion_trapecio(f,0,1,100)

1-element Array{Float64,1}:
 -1.58332

In [9]:
herramientas.integracion_simpson(f,0,1,100)

1-element Array{Float64,1}:
 -1.58333

### Test para solución de EDOs:

In [10]:
#No se me ocurrió otra cosa para evaluar los EDOs en varias dimensiones, asi que:
#Oscilador armonico amortiguado
f(x,t)=[x[2], -1.0*x[1]-x[2]]

#Graficamos [0,10] con h=0.01
list=0:0.01:10;

In [11]:
herramientas.euler(f,list,[1,0])

1001-element Array{Any,1}:
 [1,0]                    
 [1.0,-0.01]              
 [0.9999,-0.0199]         
 [0.999701,-0.0297]       
 [0.999404,-0.0394]       
 [0.99901,-0.049]         
 [0.99852,-0.0585001]     
 [0.997935,-0.0679003]    
 [0.997256,-0.0772007]    
 [0.996484,-0.0864012]    
 [0.99562,-0.0955021]     
 [0.994665,-0.104503]     
 [0.99362,-0.113405]      
 ⋮                        
 [-0.00192349,-0.00614518]
 [-0.00198494,-0.00606449]
 [-0.00204559,-0.005984]  
 [-0.00210543,-0.0059037] 
 [-0.00216446,-0.00582361]
 [-0.0022227,-0.00574373] 
 [-0.00228014,-0.00566407]
 [-0.00233678,-0.00558463]
 [-0.00239262,-0.00550541]
 [-0.00244768,-0.00542643]
 [-0.00250194,-0.00534769]
 [-0.00255542,-0.00526919]

In [12]:
herramientas.euler_pm(f,list,[1,0])

1001-element Array{Any,1}:
 [1,0]                    
 [0.99995,-0.00995]       
 [0.999801,-0.0198]       
 [0.999554,-0.02955]      
 [0.99921,-0.0392001]     
 [0.99877,-0.0487502]     
 [0.998235,-0.0582005]    
 [0.997606,-0.0675509]    
 [0.996884,-0.0768016]    
 [0.99607,-0.0859526]     
 [0.995165,-0.0950039]    
 [0.99417,-0.103956]      
 [0.993086,-0.112808]     
 ⋮                        
 [-0.00153131,-0.00622745]
 [-0.0015932,-0.00614994] 
 [-0.00165431,-0.00607259]
 [-0.00171465,-0.0059954] 
 [-0.00177422,-0.00591838]
 [-0.00183302,-0.00584155]
 [-0.00189105,-0.00576489]
 [-0.00194831,-0.00568843]
 [-0.00200482,-0.00561216]
 [-0.00206056,-0.00553609]
 [-0.00211554,-0.00546023]
 [-0.00216976,-0.00538457]

In [13]:
herramientas.euler_implicito(f,list,[1,0])  #Llevo dos horas intentandólo para n-dimensiones pero no me salió debido al Método de Newton que no acepta más de una dimensión. Sin embargo, sirve a la perfección para 1D.
#En otras palabras: en mi caso es imposible, pues estoy usando derivada numérica ya que se entendía que esto era para 1D en la tarea_12.


LoadError: DimensionMismatch("dimensions must match")

In [14]:
herramientas.runge_kutta(f,list,[1,0])

1001-element Array{Any,1}:
 [1,0]                    
 [0.99995,-0.00995]       
 [0.999801,-0.0198]       
 [0.999554,-0.02955]      
 [0.999211,-0.0392001]    
 [0.998771,-0.0487503]    
 [0.998236,-0.0582005]    
 [0.997607,-0.067551]     
 [0.996885,-0.0768017]    
 [0.996071,-0.0859527]    
 [0.995167,-0.0950041]    
 [0.994172,-0.103956]     
 [0.993088,-0.112808]     
 ⋮                        
 [-0.00153155,-0.00622848]
 [-0.00159345,-0.00615096]
 [-0.00165457,-0.0060736] 
 [-0.00171492,-0.0059964] 
 [-0.0017745,-0.00591937] 
 [-0.00183331,-0.00584252]
 [-0.00189135,-0.00576586]
 [-0.00194863,-0.00568938]
 [-0.00200514,-0.0056131] 
 [-0.00206089,-0.00553702]
 [-0.00211588,-0.00546114]
 [-0.00217012,-0.00538548]